In [3]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [4]:
# global variables
background = None
accumulated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [5]:
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [6]:
def segment(frame,threshold_min=25):
    global background
    
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret, thresh = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    image, contours, hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        # largest external contour in the ROI should be the hand
        
        hand_segment = max(contours,key=cv2.contourArea)
        
        return (thresh,hand_segment)
        

In [7]:
# count fingers with convex hull
def count_fingers(threshold_img,hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    # getting extreme points (x,y) tuples
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    center_x = (left[0] + right[0]) // 2 # x so index 0
    center_y = (top[1] + bottom[1]) // 2 # y so index 1
    
    # calculating the euclidean distance from center to all four extreme points
    distance = pairwise.euclidean_distance([center_x,center_y],Y=[left,right,top,bottom])[0]
    
    max_distance = distance.max()
    
    radius = int(0.9*max_distance)
    circumference = (2*np.pi*radius)
    
    circular_roi = np.zeros(threshold_img[:2],dtype='uint8')
    
    cv2.cricle(circular_roi,(center_x,center_y),radius,255,10)
    
    circular_roi = cv2.bitwise_and(threshold_img,threshold_img,mask=circular_roi)
    
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    count = 0
    
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # if the center of the hand is too far
        
        out_of_position = (center_y + (center_y*0.25) > (y + h))
        
        limit_points = ((circumference*0.25) > cnt.shape[0])
        
        if out_of_position and limit_points:
            count += 1
    return count

In [8]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    
    ret, frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left] # x and y splitting / cropping to get the ROI from the frame
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <=59:
            cv2.putText(frame_copy,"Wait Getting Background",(200,300),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Finger count',frame_copy)
    else:
        hand = segment(gray)
        
        if hand is not None:
            thresholded, hand_segment = hand
            
            cv2.drawContours(frame_copy,[hand_segment + (roi_right,roi_top)],-1,(255,0,0),5)
            
            fingers = count_fingers(thresholded,hand_segment)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            
            cv2.imshow("Thresholded", thresholded)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,255,0),2)
    
    num_frames += 1
    
    cv2.imshow('Finger count',frame_copy)
    
    
    k = cv2.waitKey(10) & 0xFF
    
    if k == 27:
        break
        
        
cam.release()
cv2.destroyAllWindows()
    

AttributeError: module 'sklearn.metrics.pairwise' has no attribute 'euclidean_distance'